## 기울기  
모든 변수의 편미분을 벡터로 정리한 것을 기울기(gradient)라고 함.

In [6]:
import numpy as np
def function_2(x):
    return x[0]**2 + x[1]**2
def numerical_gradient(f, x):
    h= 1e-4
    grad = np.zeros_like(x)#x같은 0의 자료를 만들어줌
    
    for i in range(x.size):
        tmp_val = x[i]
        x[i] = tmp_val+h
        fxh1 = f(x)
        
        x[i] = tmp_val-h
        fxh2 = f(x)
        
        grad[i] = (fxh1-fxh2)/(2*h)
        x[i] = tmp_val
    return grad

In [7]:
numerical_gradient(function_2, np.array([3.0,4.0]))

array([6., 8.])

In [10]:
numerical_gradient(function_2, np.array([3.0,0.0]))

array([6., 0.])

기울기는 각 지점에서 낮아지는 방향을 가리킴

-> 기우리가 가리키는 방향은 각 장소에서 함수의 출력 값을 가장 크게 줄이는 방향

## 경사하강법

기울기가 가라키는 방향으로 가야지만 값이 줄어들기 때문에 경사법은  
기울기를 따라 기계학습을 최적화 시키하는 방법

 $$\eta$$는 에타

In [12]:
def gradient_descent(f, init_x, lr = 0.01, step_num = 100):
    x = init_x
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        x-=lr*grad
    return x

In [13]:
def function_2(x):
    return x[0]**2 + x[1]**2

init_x = np.array([-3.0,4.0])
gradient_descent(function_2, init_x = init_x, lr = 0.1, step_num=100)


array([-6.11110793e-10,  8.14814391e-10])

In [15]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=10.0, step_num=100)

array([-2.58983747e+13, -1.29524862e+12])

In [16]:
init_x = np.array([-3.0, 4.0])
gradient_descent(function_2, init_x=init_x, lr=1e-10, step_num=100)

array([-2.99999994,  3.99999992])

학습률이 너쿠 크면 발상해버림, 너무 작으면 끝까지 갱신 x

## 신경망에서의 기울기

In [30]:
def _numerical_gradient_1d(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    for idx in range(x.size):
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        
    return grad


def numerical_gradient_2d(f, X):
    if X.ndim == 1:
        return _numerical_gradient_1d(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = _numerical_gradient_1d(f, x)
        
        return grad


def numerical_gradient(f, x):
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        idx = it.multi_index
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + h
        fxh1 = f(x) # f(x+h)
        
        x[idx] = tmp_val - h 
        fxh2 = f(x) # f(x-h)
        grad[idx] = (fxh1 - fxh2) / (2*h)
        
        x[idx] = tmp_val # 값 복원
        it.iternext()   
        
    return grad


def identity_function(x):
    return x


def step_function(x):
    return np.array(x > 0, dtype=np.int)


def sigmoid(x):
    return 1 / (1 + np.exp(-x))    


def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)
    

def relu(x):
    return np.maximum(0, x)


def relu_grad(x):
    grad = np.zeros(x)
    grad[x>=0] = 1
    return grad
    

def softmax(x):
    if x.ndim == 2:
        x = x.T
        x = x - np.max(x, axis=0)
        y = np.exp(x) / np.sum(np.exp(x), axis=0)
        return y.T 

    x = x - np.max(x) # 오버플로 대책
    return np.exp(x) / np.sum(np.exp(x))


def mean_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)


def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 원-핫 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size


def softmax_loss(X, t):
    y = softmax(X)
    return cross_entropy_error(y, t)


In [33]:
# coding: utf-8
import sys, os
sys.path.append(os.pardir)  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
import numpy as np



class simpleNet:
    def __init__(self):
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss


In [34]:
def f(W):
    return net.loss(x, t)
dw = numerical_gradient(f, net.W)
print(dw)

[[ 0.21275089  0.19893766 -0.41168855]
 [ 0.31912634  0.29840649 -0.61753283]]


In [35]:
f = lambda w: net.loss(x, t)
dW =numerical_gradient(f, net.W)

## 신경망 학습의 4단계  
1단계 - 미니배치: 훈련 데이터 중 일부를 무작위로 선별하여 이 미니배치의 손실 함수 값을 줄이는 것이 목표.  
2단계 - 기울기 산출: 미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개변수의 기울기를 구하고, 이 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시함.  \
3단계 - 매개변수 갱신: 가중치 매개변수를 기울기 방향으로 조금 갱신  
4단계 - 반복

In [40]:
import sys, os
sys.path.append(os.pardir)
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1,b2 = self.params['b1'], self.params['b2']
        
        a1 = np.dot(x, W1) +b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) +b2
        y = softmax(a2)
        
        return y
    def loss(self, x, t):
        y = self.predict(x)
        return cross_entropy_error(y, t)
    def accuracy(self, x, y):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / loat(x.shape[0])
        return accuracy
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads={}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        return grads

In [41]:
net= TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
net.params['W1'].shape



(784, 100)

In [42]:
x = np.random.rand(100, 784)
t = np.random.rand(100, 10)
grads = net.numerical_gradient(x, t)

grads['W1'].shape

(784, 100)